# 5. Árboles de Decisión

Veremos árboles de decisión y los conceptos subyacentes asociados.

Haremos ejemplos de juguete y con datos generados artificialmente.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Entropía y Ganancia de Información

### Entropía

Definamos entropía para una distribución probabilista:

$$H(Y) = - \sum_{i=1}^k P(Y = y_i) log_2 P(Y = y_i)$$

In [ ]:
def entropy(probs):
    return - np.sum(probs * np.log2(probs))

Veamos posibles entropías para el problema de tirar una moneda adulterada:

In [ ]:
entropy(np.array([0.5, 0.5]))

In [ ]:
entropy(np.array([0.01, 0.99]))

In [ ]:
X = np.linspace(0, 1)[1:-1]
plt.plot(X, [entropy([x, 1-x]) for x in X])
plt.xlabel('P(Y=y_1)')
plt.ylabel('entropy')
plt.show()

En el caso de dos monedas, tenemos cuatro combinaciones posibles, siendo $0.25$ la probabilidad de cada evento. Ejemplos:

In [ ]:
entropy(np.array([0.25, 0.25, 0.25, 0.25]))

Pero si las monedas estan sesgadas tendríamos: 

In [ ]:
entropy(np.array([0.49, 0.49, 0.01, 0.01]))

### Entropía de un Dataset

Un dataset define una distribución empírica. La entropía del dataset es entones la entropía de la distribución asociada. Definamos el cálculo de la distribución, y luego redefinamos entropía:

In [ ]:
def probs(y):
    _, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return probs

Definimos un dataset de 6 elementos con dos atributos $X_1$ y $X_2$ con dos tipos de etiquetas: 

$False: -1$ y $True: 1$. 
<img src="img/tablaDataset2.png" width="100"/>
Siendo las etiquetas resultantes del mismo: 


In [ ]:
y = np.array([1, 1, 1, 1, 1, -1])

Mediante la función "probs" podemos calcular la probilidad de cada clase en este dataset: 

In [ ]:
probs(y)

Esos resultados nos informan que hay un desbalance en el mismo (hay más probabilidad de una de las clases).

Ahora obtengamos la entropía del dataset:

In [ ]:
def entropy(y):
    p = probs(y)
    return - np.sum(p * np.log2(p))

In [ ]:
entropy(y)

### Entropía Condicional

Definamos entropía condicional:

$$H(Y|X) = - \sum_{j=1}^v P(X = x_j) \sum_{i=1}^k P(Y = y_i | X = x_j) log_2 P(Y = y_i | X = x_j)$$

Equivalentemente, 

$$H(Y|X) = \sum_{j=1}^v P(X = x_j) H(Y|X = x_j)$$

Tomaremos $X$ binaria ($v=2$), por lo que la entropía condicional tendrá sólo dos términos.


In [ ]:
def cond_entropy(y1, y2):
    size = y1.shape[0] + y2.shape[0]
    return y1.shape[0] / size * entropy(y1) + y2.shape[0] / size * entropy(y2) 

Esta función toma como argumento los dos subconjuntos de datos que se forman al considerar una de las variables como condición.

Si analizámos la varibale $Y$ donde la $X_1$ es $True$, tenemos: 

In [ ]:
y_X1true = np.array([1,1,1,1]) #X1 = True

de igual manera, la varibale $Y$  donde la $X_1$ es $False$, tenemos: 

In [ ]:
y_X1false =  np.array([1,-1]) #X1 = False

Haciendo uso de la función "cond_entropy" podemos obtener la entropía condicional de la variable $Y$ dado $X_1$

In [ ]:
cond_entropy(y_X1true,y_X1false)  # x1

De igual manera podemos calcular la entropía condicional dado la variable $X_2$

In [ ]:
y_X2true = np.array([1,1,1]) #X2 = True
y_X2false = np.array([1,1,-1]) #X2 = False
cond_entropy(y_X2true, y_X2false)  # x2

### Ganancia de Información

La ganancia de información será simplemente la diferencia entre entropía y entropía condicional:

In [ ]:
def information_gain(y1, y2):
    y = np.concatenate((y1,y2))
    return entropy(y) - cond_entropy(y1,y2)

Podemos calcular la ganancia de información que implicaría dividir el dataset tomando como $X_1$ como nodo referencia. Para ello hacemos uso de la función "information_gain" que toma los dos subconjuntos de etiquetas que se formarían al tomar una u otra variable como referencia. 

Para el caso de $X_1$: 

In [ ]:
information_gain(y_X1true, y_X1false)  # x1

Para el caso de $X_2$: 

In [ ]:
information_gain(y_X2true, y_X2false)  # x2

## Datos Sintéticos No Linealmente Separables

Haremos algunos experimentos con datos generados sintéticamente. Estos datos serán no linealmente separables.

Ejemplos típicos de datos no linealmente separables son los de tipo "OR", "AND" y "XOR". Usaremos datos de tipo "OR" para este ejemplo.



In [ ]:
size = 200

In [ ]:
np.random.seed(0)
X = np.random.randn(size, 2)
y_true = np.logical_or(X[:, 0] > 0, X[:, 1] > 0)    # datos "OR"
#y_true = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)  # datos "XOR"
#y_true = np.logical_and(X[:, 0] > 0, X[:, 1] > 0)  # datos "AND"
y_true = y_true.astype(int)
y_true[y_true == 0] = -1

In [ ]:
plt.scatter(X[y_true==1, 0], X[y_true==1, 1], color="royalblue", label="1")
plt.scatter(X[y_true==-1, 0], X[y_true==-1, 1], color="red", label="-1")
plt.grid()
plt.xlabel("X[0]")
plt.ylabel("X[1]")
plt.legend()
plt.show()

### División en Entrenamiento y Evaluación

Separemos la mitad para entrenamiento y la otra para evaluación.

In [ ]:
train_size = 50
test_size = size - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y_true[:train_size], y_true[train_size:]

In [ ]:
X_train.shape, X_test.shape

### Clasificación Lineal

Veamos qué tan mal anda un clasificador lineal sobre estos datos.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train);

In [ ]:
from utils import plot_decision_boundary
plt.xlabel("X[0]")
plt.ylabel("X[1]")
plot_decision_boundary(lambda x: model.predict(x), X, y_true)

Calculemos la calidad de la predicción en entrenamiento y evaluación:

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy: {test_acc:0.2}')

### Nota al Margen: Induciendo Separabilidad Lineal

Muchas veces se pueden convertir datos no linealmente separables en datos separables (o casi) mediante la introducción de nuevos atributos que combinan los atributos existentes.
Un ejemplo de estos son los atributos polinomiales.

Aquí lo haremos con datos "OR", pero la diferencia es mucho más notable con datos de tipo "XOR".

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

pre = PolynomialFeatures(
    degree=2,
    interaction_only=True,  # para usar solo x0*x1, no x0*x0 ni x1*x1
    include_bias=False)
X_train2 = pre.fit_transform(X_train)

In [ ]:
X_train.shape, X_train2.shape  # se agregó el feature x0*x1

Grafiquemos:

In [ ]:
plt.scatter(X_train2[y_train==1, 1], X_train2[y_train==1, 2], color="dodgerblue", edgecolors='k', label="1")
plt.scatter(X_train2[y_train==-1, 1], X_train2[y_train==-1, 2], color="tomato", edgecolors='k', label="-1")
plt.grid()
plt.legend()
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    PolynomialFeatures(degree=2, interaction_only=True, include_bias=False),
    LogisticRegression()
)
model.fit(X_train, y_train);

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy: {test_acc:0.2}')

In [ ]:
from utils import plot_decision_boundary

plot_decision_boundary(lambda x: model.predict(x), X, y_true)
plt.xlabel("X[0]")
plt.ylabel("X[1]")

### Entropía y Valores Reales

Calculemos la entropía inicial, y veamos cómo condicionar la entropía sobre variables reales (i.e. no categóricas)

In [ ]:
y_train

Podemos calcular la probabilidad de cada clase:

In [ ]:
probs(y_train)

y la entropía:

In [ ]:
entropy(y_train)

Para hacer una división sobre una variable real usaremos un valor "threshold" (umbral):

In [ ]:
def split(X, y, i, threshold):
    y1 = y[X[:, i] > threshold]
    y2 = y[X[:, i] <= threshold]
    return y1, y2

definimos la "split" que toma como argumento el dataset con sus etiquetas, el indice de la variable que usaremos para dividir el dataset y el umbral para realizarlo y nos devuelve dos subconjuntos de este dataset.

Si utilizamos la variable $X_1$ con un umbral de $0.00$ obtenemos:

In [ ]:
y1, y2 = split(X_train, y_train, 0, 0.00)

In [ ]:
y1,y2

Siendo la entropía de cada uno de ellos:

In [ ]:
entropy(y1), entropy(y2)

In [ ]:
cond_entropy(y1, y2)

In [ ]:
information_gain(y1,y2)

### Buscando la Mejor División

Ilustraremos un paso en la construcción del árbol de decisión.

Probemos muchos threshold para ambas variables y seleccionemos la mejor división.

En este caso buscaremos en una grilla uniforme de valores, pero existen técnicas mejores.

In [ ]:
np.linspace(-2.5, 2.5, 11)

In [ ]:
best_ig = 0

for i in [0, 1]:
    for threshold in np.linspace(-2.5, 2.5, 11):
        y1, y2 = split(X_train, y_train, i, threshold)
        ig = information_gain(y1, y2)
        print(f'i={i}\tthreshold={threshold:+00.2f}\tig={ig:.2f}')

        if ig >= best_ig:
            best_ig = ig
            best_feature = i
            best_threshold = threshold

print('Mejor división:')
print(f'feature={best_feature}, threshold={best_threshold}, ig={best_ig:00.2f}')


Dividamos los datos de acuerdo a esta frontera:

In [ ]:
best_feature, best_threshold

Podemos ver los datos en el gráfico

In [ ]:
plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], color="royalblue", label="1")
plt.scatter(X_train[y_train==-1, 0], X_train[y_train==-1, 1], color="red", label="-1")

plt.axhline(y=0, xmin=-3, xmax=3, color='green', linestyle='-.', linewidth=2)

plt.xlabel("X[0]")
plt.ylabel("X[1]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
y1, y2 = split(X_train, y_train, best_feature, best_threshold)

Con esta división, la entropía baja considerablemente:

In [ ]:
entropy(y_train)

In [ ]:
cond_entropy(y1, y2)

## Árbol de Decisión con Scikit-learn

Aprendamos un árbol de decisión usando scikit-learn. Para ello usaremos la clase [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html):


In [ ]:
from sklearn.tree import DecisionTreeClassifier
#DecisionTreeClassifier?
clf = DecisionTreeClassifier(max_depth=8, criterion='entropy', random_state=0)
#clf = DecisionTreeClassifier(criterion='entropy', random_state=0)

clf.fit(X_train, y_train)

Ahora predecimos y evaluamos:

In [ ]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy: {test_acc:0.2}')

Dibujamos la frontera de decisión:

In [ ]:
from utils import plot_decision_boundary

plot_decision_boundary(lambda x: clf.predict(x), X_train, y_train)
plt.xlabel("X[0]")
plt.ylabel("X[1]")

También podemos inspeccionar el árbol:

In [ ]:
from sklearn.tree import plot_tree

plot_tree(clf,filled=True);

In [ ]:
entropy(y_train)

## Ejercicios

1. Probar todos los experimentos con un dataset de tipo "XOR". ¿Qué sucede al decidir la división en el primer nivel del árbol?

## Referencias

Scikit-learn:

- [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
- [User Guide: Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
